In [68]:
import nltk
import random 
import string
import numpy as np
from nltk.corpus import names

# Clasificacion de palabras(Nombres)

## Attribute extraction

In [104]:
a = np.arange(-10,10)

In [125]:
def attribute(word):
    attr = {}
    attr['last_word'] = word[-1].lower()
    attr['lats_3_words'] = word[-4:].lower()
    for letra in string.ascii_lowercase:
        attr[f'{letra} in'] = np.where((letra in word),1,0).tolist()
    return attr

In [126]:
tagset = [(name,'M') for name in names.words('male.txt')] + [(name,'F') for name in names.words('female.txt')]

In [127]:
random.shuffle(tagset)

In [128]:
tagset[:10]

[('Wittie', 'M'),
 ('Flossy', 'F'),
 ('Stanton', 'M'),
 ('Clarie', 'F'),
 ('Eldon', 'M'),
 ('Emmy', 'M'),
 ('Wynne', 'F'),
 ('Norah', 'F'),
 ('Gian', 'M'),
 ('Dionysus', 'M')]

In [129]:
fset = [(attribute(name),g) for (name,g) in tagset]

In [130]:
train = fset[:6355]
test = fset[6355:]

## Clasificador

In [131]:
classifier = nltk.NaiveBayesClassifier.train(train)

## Testing the algorith

In [132]:
classifier.classify(attribute('Jose'))

'M'

In [133]:
nltk.classify.accuracy(classifier,test)

0.7816236626809314

# Clasificacion de documentos (Spam)

In [156]:
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
#import zipfile

In [142]:
df = pd.read_csv('../data/datasets/email/csv/spam-apache.csv', names=['clase','contenido'])

In [143]:
df

,clase,contenido
0,-1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
1,1,> Russell Turpin:\n> > That depends on how the...
2,-1,Help wanted. We are a 14 year old fortune 500...
3,-1,Request A Free No Obligation Consultation!\nAc...
4,1,Is there a way to look for a particular file o...
...,...,...
245,1,> From: Paul Linehan [mailto:plinehan@yahoo.co...
246,-1,REQUEST FOR URGENT BUSINESS ASSISTANCE\n------...
247,1,> From: fork-admin@xent.com [mailto:fork-admin...
248,-1,Email marketing works! There's no way around ...


## Attribute extraction

In [149]:
df['tokens'] = df['contenido'].apply(lambda x : word_tokenize(x))

In [160]:
alll_words = nltk.FreqDist([w for token in df['tokens'] for w in token if w not in stopwords.words('english')])
most_common = alll_words.most_common(200)

In [171]:
def document_attribute(document):
    attr = {}
    document_vocabulary = set(document)
    for word , _ in most_common:
        attr[f'Have {word}'] = np.where(word in document_vocabulary, 1, 0).tolist()
    return attr

In [188]:
# aplicar los atributos y dividir la data en entrenamineto y test
fset = [(document_attribute(w),v) for (w,v) in zip(df['tokens'],df['clase'])]
n = int(len(df['tokens'])*80/100)
train , test = fset[:n],fset[n:]

## Modelo

In [192]:
classifier = nltk.NaiveBayesClassifier.train(train)

In [193]:
nltk.classify.accuracy(classifier,test)

0.9